In [ ]:
#FETCH ALL THE URLS
import time
import re
from selenium import webdriver
import csv
from bs4 import BeautifulSoup as bs


browser = webdriver.Chrome(r'C:\Users\Reema Mehta\Downloads\chromedriver_win32\chromedriver.exe')

# Tell Selenium to get the URL you're interested in.
browser.get("https://statestreet.wd1.myworkdayjobs.com/Global")
# browser.implicitly_wait(100)

# Selenium script to scroll to the bottom, wait 3 seconds for the next batch of data to load, then continue scrolling.  It will continue to do this until the page stops loading new data.
lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match=False
while(match==False):
    lastCount = lenOfPage
    time.sleep(3)
    lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    if lastCount==lenOfPage:
#        print('lastCount and lenOfPage matched: ',lenOfPage)
        match=True

# Now that the page is fully scrolled, grab the source code.
source_data = browser.page_source

# Throw your source into BeautifulSoup and start parsing!
bs_data = bs(source_data, "lxml")
#print(bs_data)

base_url = 'https://statestreet.wd1.myworkdayjobs.com/en-US/Global/job/'
test_links = bs_data.find_all('li',attrs={'class':'WIYF WK3N WE5 WP-F'})
# my_str = "hey th~!ere"
# my_new_string = re.sub('[^a-zA-Z0-9 \n\.]', '', my_str)
# print my_new_string
link_url =[]

for link in test_links:
    designation = link.find('div',attrs={'class':'gwt-Label WOTO WISO'})
    address = link.find('span', attrs={'class':'gwt-InlineLabel WM-F WLYF'})
#     print('designation is: ', designation.get_text())
#     print('address is: ', address.get_text())
    address_label = address.get_text()
    title = designation.get_text()

    address_label_contents = address_label.split('|')
#    print(address_label_contents)
#    print(type(address_label_contents))
    a = len(address_label_contents)
#    print(a)
    if a ==3:
        location_more = address_label_contents[0].strip()
        #        print(location_more)
        if location_more.count(',')>1:
            location_trim = location_more.rsplit(',',1)
            location = location_trim[0]
        else:
            location = location_more
        location = re.sub('[^a-zA-Z0-9-]+','-',location)
#        print(location)
        title = re.sub('[^a-zA-Z0-9-]','-',title)
        job_id = address_label_contents[1].strip()
#     location.replace(',','-')
#     print(location, job_id)
        if len(address_label_contents)==2:
            url = base_url+title+'_'+address_label_contents[0].strip()
        else:
            url = base_url+location+'/'+title+'_'+job_id
            link_url.append(url)
#        print(url)
browser.close()

In [15]:
# APPROACH 2 : FOR ALL LINKS

all_contents = []
link_url =[]

for joburl in link_url:
    #print(joburl)
    browser = webdriver.Chrome(r'C:\Users\Reema Mehta\Downloads\chromedriver_win32\chromedriver.exe')
    browser.get(joburl)
    # Selenium script to scroll to the bottom, wait 3 seconds for the next batch of data to load, then continue scrolling.  It will continue to do this until the page stops loading new data.
    lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match=False
    while(match==False):
        lastCount = lenOfPage
        time.sleep(3)
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount==lenOfPage:
    #        print('lastCount and lenOfPage matched: ',lenOfPage)
            match=True

    # Now that the page is fully scrolled, grab the source code.
    source_data = browser.page_source
    
    # Throw your source into BeautifulSoup and start parsing!
    bs_data = bs(source_data, "lxml")

    job_descrp = bs_data.find_all('div',attrs={'class':'GWTCKEditor-Disabled'}) #extracts all the text in this tag

    jobDescList = []

    for job in job_descrp:
        jobDescList.append(job.get_text())

    # print(len(jobDescList))
    # print(jobDescList[1])
    browser.close()

    string = ""

    for i in range(0,(len(jobDescList)-1)):
        string = string+jobDescList[i]

    all_contents.append(string)



In [16]:
import collections

def get_count(word_count_tuple):
    return word_count_tuple[1]

word_counts = collections.Counter(all_contents)

# This function sorts the words based on the number of times repeated and saves them in a csv file
def print_top():
    word_count = word_counts
    items = sorted(word_count.items(),key=get_count,reverse=True)
    with open(r'C:\Users\Reema Mehta\Downloads\jobdesc_SS.csv','w') as out:
        csv_out=csv.writer(out)
        #csv_out.writerow('Name')
        for row in items:
            csv_out.writerow(row)
print_top()


In [14]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize,sent_tokenize
import collections
from tika import parser
import csv
import re

print("ALL CONTENTS")
print(all_contents[])

stop_words = set(stopwords.words('english'))


i=0
for data in all_contents:
    word_tokens1 = word_tokenize(data)
    words1 = [w for w in word_tokens1 if w.isalpha()]
    filtered_sentence = []
    i = i+1
    print("DATA", i)
    for w in words1: 
        #w.lower()
        #print(w.lower())
        if w.lower() not in stop_words: 
            filtered_sentence.append(w)
            #print('Total Words :',len(filtered_sentence))
    #print(filtered_sentence)
    
    word_counts = collections.Counter(filtered_sentence)

# It sorts the words xbased on the number of times repeated and saves them in a csv file
def print_top():
    word_count = word_counts
    items = sorted(word_count.items(),key=get_count,reverse=True)
    with open(r'C:\Users\Reema Mehta\Desktop\DS\wordss.csv','w') as out:
        csv_out=csv.writer(out)
        for row in items:
            csv_out.writerow(row)
#    for item in items[:]:
#        (item[0],item[1])
#        print(type(item))
#        print(item[0],item[1])
#        print (item[0], 'has been repeated ', item[1],' times')
#print(type(item))

print_top()

SyntaxError: invalid syntax (<ipython-input-14-0e7b4da927fe>, line 9)

In [ ]:
list = [ 'abcd', 786 , 2.23, 'john', 70.2 ]
tinylist = [123, 'john']
 
 #comparing value of two list
for item in list:
	for item1 in tinylist:
		if item == item1:
			print item
		else:
			print "No match"
 
print "****************\n"